In [1]:
#pip install alpaca-trade-api 

In [2]:
# Initial imports
import os
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
from alpaca_trade_api.rest import REST, TimeFrame
import pandas as pd
# Load .env environment variables
load_dotenv()

True

In [3]:
import alpaca_trade_api as tradeapi

API_KEY = os.getenv("ALPACA_API_KEY")
API_SECRET = os.getenv("ALPACA_SECRET_KEY")
ALPACA_API_BASE_URL = "https://paper-api.alpaca.markets"
ALPACA_PAPER = True

In [4]:
 # Create a connection to the API 
api = REST(key_id=API_KEY,secret_key=API_SECRET,base_url="https://paper-api.alpaca.markets")

### View Positions ###

### Read in BTC Data from ALPACA API ###

In [5]:
# Fetch 1Minute historical bars of Bitcoin
btc_df = api.get_crypto_bars("BTCUSD", TimeFrame.Minute).df

# Filter data by exchange
btc_df = btc_df[btc_df.exchange == 'CBSE']

#Display btc data
btc_df.head()

,exchange,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,,
2022-12-03 06:00:00+00:00,CBSE,17018.76,17020.76,17015.73,17017.14,9.255227,199,17018.691648
2022-12-03 06:01:00+00:00,CBSE,17017.14,17018.46,17015.88,17017.44,3.243799,189,17016.858915
2022-12-03 06:02:00+00:00,CBSE,17017.43,17018.89,17014.16,17015.45,6.045559,171,17015.832465
2022-12-03 06:03:00+00:00,CBSE,17015.19,17018.35,17014.53,17017.00,4.641381,126,17016.129595
2022-12-03 06:04:00+00:00,CBSE,17016.54,17016.97,17014.25,17014.66,2.316787,127,17015.166159


### Clean data and calculate Bitcoin daily returns ### 

In [6]:
# Filter the date index and close columns
btc_signals_df =  btc_df.loc[:, ["close"]]

# Use the pct_change function to generate  returns from close prices
btc_signals_df["Actual Returns"] = btc_df["close"].pct_change()

# Drop all NaN values from the DataFrame
btc_signals_df = btc_signals_df.dropna()

# Review the DataFrame
display(btc_signals_df.head())
display(btc_signals_df.tail())


,close,Actual Returns
timestamp,,
2022-12-03 06:01:00+00:00,17017.44,0.000018
2022-12-03 06:02:00+00:00,17015.45,-0.000117
2022-12-03 06:03:00+00:00,17017.00,0.000091
2022-12-03 06:04:00+00:00,17014.66,-0.000138
2022-12-03 06:05:00+00:00,16999.15,-0.000912


,close,Actual Returns
timestamp,,
2022-12-04 03:49:00+00:00,16960.50,-0.000085
2022-12-04 03:50:00+00:00,16960.41,-0.000005
2022-12-04 03:51:00+00:00,16957.48,-0.000173
2022-12-04 03:52:00+00:00,16959.50,0.000119
2022-12-04 03:53:00+00:00,16958.38,-0.000066


### Add SMA trading strategy ###

In [7]:

# Step 2: Generate trading signals using short- and long-window and SMA_200 SMA values

# Set the short window and long window
short_window = 10
long_window = 100
sma_200 = 200

# Generate the fast and slow simple moving averages (10 and 100 days, respectively)
btc_signals_df['SMA_Fast'] = btc_signals_df['close'].rolling(window=short_window).mean()
btc_signals_df['SMA_Slow'] = btc_signals_df['close'].rolling(window=long_window).mean()
btc_signals_df['SMA_200'] = btc_signals_df['close'].rolling(window=sma_200).mean()

btc_signals_df = btc_signals_df.dropna()

# Review the DataFrame
display(btc_signals_df.head())
display(btc_signals_df.tail())

,close,Actual Returns,SMA_Fast,SMA_Slow,SMA_200
timestamp,,,,,
2022-12-03 09:20:00+00:00,16962.79,0.000200,16964.241,16959.3515,16965.95840
2022-12-03 09:21:00+00:00,16958.98,-0.000225,16963.230,16959.4105,16965.66610
2022-12-03 09:22:00+00:00,16959.55,0.000034,16962.403,16959.4705,16965.38660
2022-12-03 09:23:00+00:00,16961.95,0.000142,16961.992,16959.5088,16965.11135
2022-12-03 09:24:00+00:00,16961.50,-0.000027,16961.620,16959.5006,16964.84555


,close,Actual Returns,SMA_Fast,SMA_Slow,SMA_200
timestamp,,,,,
2022-12-04 03:49:00+00:00,16960.50,-0.000085,16967.280,16971.6775,16968.5652
2022-12-04 03:50:00+00:00,16960.41,-0.000005,16965.858,16971.7114,16968.6783
2022-12-04 03:51:00+00:00,16957.48,-0.000173,16964.423,16971.7397,16968.7769
2022-12-04 03:52:00+00:00,16959.50,0.000119,16963.137,16971.7392,16968.8796
2022-12-04 03:53:00+00:00,16958.38,-0.000066,16962.208,16971.7273,16968.9638


### Initiate Signals ###

In [8]:
# Initialize the new Signal column
btc_signals_df['Signal'] = 0.0

# When Actual Returns are greater than or equal to 0, generate signal to buy stock long
btc_signals_df.loc[(btc_signals_df['Actual Returns'] >= 0), 'Signal'] = 1

# When Actual Returns are less than 0, generate signal to sell stock short
btc_signals_df.loc[(btc_signals_df['Actual Returns'] < 0), 'Signal'] = -1

# Review the DataFrame
display(btc_signals_df.head())
display(btc_signals_df.tail())

,close,Actual Returns,SMA_Fast,SMA_Slow,SMA_200,Signal
timestamp,,,,,,
2022-12-03 09:20:00+00:00,16962.79,0.000200,16964.241,16959.3515,16965.95840,1.0
2022-12-03 09:21:00+00:00,16958.98,-0.000225,16963.230,16959.4105,16965.66610,-1.0
2022-12-03 09:22:00+00:00,16959.55,0.000034,16962.403,16959.4705,16965.38660,1.0
2022-12-03 09:23:00+00:00,16961.95,0.000142,16961.992,16959.5088,16965.11135,1.0
2022-12-03 09:24:00+00:00,16961.50,-0.000027,16961.620,16959.5006,16964.84555,-1.0


,close,Actual Returns,SMA_Fast,SMA_Slow,SMA_200,Signal
timestamp,,,,,,
2022-12-04 03:49:00+00:00,16960.50,-0.000085,16967.280,16971.6775,16968.5652,-1.0
2022-12-04 03:50:00+00:00,16960.41,-0.000005,16965.858,16971.7114,16968.6783,-1.0
2022-12-04 03:51:00+00:00,16957.48,-0.000173,16964.423,16971.7397,16968.7769,-1.0
2022-12-04 03:52:00+00:00,16959.50,0.000119,16963.137,16971.7392,16968.8796,1.0
2022-12-04 03:53:00+00:00,16958.38,-0.000066,16962.208,16971.7273,16968.9638,-1.0


### Historic Strategy Returns ###

In [9]:
# Calculate the strategy returns and add them to the signals_df DataFrame
btc_signals_df['Strategy Returns'] = btc_signals_df['Actual Returns'] * btc_signals_df['Signal'].shift()
btc_signals_df.drop_duplicates()

# Review the DataFrame
display(btc_signals_df.head())
display(btc_signals_df.tail())

,close,Actual Returns,SMA_Fast,SMA_Slow,SMA_200,Signal,Strategy Returns
timestamp,,,,,,,
2022-12-03 09:20:00+00:00,16962.79,0.000200,16964.241,16959.3515,16965.95840,1.0,NaN
2022-12-03 09:21:00+00:00,16958.98,-0.000225,16963.230,16959.4105,16965.66610,-1.0,-0.000225
2022-12-03 09:22:00+00:00,16959.55,0.000034,16962.403,16959.4705,16965.38660,1.0,-0.000034
2022-12-03 09:23:00+00:00,16961.95,0.000142,16961.992,16959.5088,16965.11135,1.0,0.000142
2022-12-03 09:24:00+00:00,16961.50,-0.000027,16961.620,16959.5006,16964.84555,-1.0,-0.000027


,close,Actual Returns,SMA_Fast,SMA_Slow,SMA_200,Signal,Strategy Returns
timestamp,,,,,,,
2022-12-04 03:49:00+00:00,16960.50,-0.000085,16967.280,16971.6775,16968.5652,-1.0,-0.000085
2022-12-04 03:50:00+00:00,16960.41,-0.000005,16965.858,16971.7114,16968.6783,-1.0,0.000005
2022-12-04 03:51:00+00:00,16957.48,-0.000173,16964.423,16971.7397,16968.7769,-1.0,0.000173
2022-12-04 03:52:00+00:00,16959.50,0.000119,16963.137,16971.7392,16968.8796,1.0,-0.000119
2022-12-04 03:53:00+00:00,16958.38,-0.000066,16962.208,16971.7273,16968.9638,-1.0,-0.000066


### Pull Account data from Alpaca ###

In [10]:
# get our account data from alpaca 
account = api.get_account()
#Display Account Data
account

Account({   'account_blocked': False,
    'account_number': 'PA3YR0B1DXK2',
    'accrued_fees': '0',
    'balance_asof': '2022-12-02',
    'bod_dtbp': '0',
    'buying_power': '1878114.92617532268',
    'cash': '939057.46308766134',
    'created_at': '2022-11-24T09:23:13.041152Z',
    'crypto_status': 'ACTIVE',
    'crypto_tier': 0,
    'currency': 'USD',
    'daytrade_count': 0,
    'daytrading_buying_power': '0',
    'effective_buying_power': '1878114.92617532268',
    'equity': '939057.46308766134',
    'id': 'fab3a39a-30c4-4f96-8698-c62bf476488f',
    'initial_margin': '0',
    'last_equity': '1018885.10899616125',
    'last_maintenance_margin': '0',
    'long_market_value': '0',
    'maintenance_margin': '0',
    'multiplier': '2',
    'non_marginable_buying_power': '937057.46',
    'pattern_day_trader': False,
    'pending_transfer_in': '0',
    'portfolio_value': '939057.46308766134',
    'position_market_value': '0',
    'regt_buying_power': '1878114.92617532268',
    'short_ma

### Determine BTC portfolio weight & buying power ###

In [11]:

account.equity
equity = account.equity
equity = float(equity)
btc_portfolio_weight = 0.30 * equity

non_marginable_buying_power = account.non_marginable_buying_power
non_marginable_buying_power= float(non_marginable_buying_power)
non_marginable_buying_power

937057.46

In [12]:
btc_signals_df['non_marginable_buying_power'] = non_marginable_buying_power
btc_signals_df.tail()

,close,Actual Returns,SMA_Fast,SMA_Slow,SMA_200,Signal,Strategy Returns,non_marginable_buying_power
timestamp,,,,,,,,
2022-12-04 03:49:00+00:00,16960.50,-0.000085,16967.280,16971.6775,16968.5652,-1.0,-0.000085,937057.46
2022-12-04 03:50:00+00:00,16960.41,-0.000005,16965.858,16971.7114,16968.6783,-1.0,0.000005,937057.46
2022-12-04 03:51:00+00:00,16957.48,-0.000173,16964.423,16971.7397,16968.7769,-1.0,0.000173,937057.46
2022-12-04 03:52:00+00:00,16959.50,0.000119,16963.137,16971.7392,16968.8796,1.0,-0.000119,937057.46
2022-12-04 03:53:00+00:00,16958.38,-0.000066,16962.208,16971.7273,16968.9638,-1.0,-0.000066,937057.46


### Determine Quantity and $ value of Trade ###

In [13]:
#Pull last BTC Closing Price
btc_price = btc_signals_df['close'][-1]
btc_price

#Intialize quantity and value of trade
QTY_PER_TRADE = btc_portfolio_weight / btc_price
order_value = QTY_PER_TRADE * btc_price
order_value = float(order_value)
sell_qty_per_trade = 5
usd_price_per_trade = QTY_PER_TRADE * btc_price
print(f'The quantity of BTC per trade is {QTY_PER_TRADE} at a USD $ value of {usd_price_per_trade} with the current price of BTC at ${btc_price}') 

The quantity of BTC per trade is 16.612273042961554 at a USD $ value of 281717.2389262984 with the current price of BTC at $16958.38


### Submit Buy Order & display tranaction details ###

In [14]:
# Create a market order to buy 1 Bitcoin
if order_value <= non_marginable_buying_power:
    order_buy = api.submit_order('BTCUSD', qty=QTY_PER_TRADE, side='buy',
                            time_in_force='gtc')
    # get our account data from alpaca 
    account = api.get_account()
    # Get current position on Bitcoin. Yields error if we don't hold any
    api.get_position('BTCUSD')

    # Get all positions and find out if we hold any Bitcoin
    positions = api.list_positions()
    position_qty = 0
    for p in positions:
        if p.symbol == 'BTCUSD':
            position_qty = float(p.qty)
        
    order_value = QTY_PER_TRADE * btc_price
    non_marginable_buying_power = account.non_marginable_buying_power
    #Display Transaction details
    print(f'Updated Position: {position_qty} / Order Value from position: $ {order_value} / Portfolio buying power is now : ${non_marginable_buying_power}')
    print(f'Buy Order Transation details:')
    print(order_buy)
else: 
        print((' Side: Hold'))



Updated Position: 16.570742359 / Order Value from position: $ 281717.2389262984 / Portfolio buying power is now : $655325.27
Buy Order Transation details:
Order({   'asset_class': 'crypto',
    'asset_id': '276e2673-764b-4ab6-a611-caf665ca6340',
    'canceled_at': None,
    'client_order_id': 'ed8914c7-4fdf-4271-aaaa-d03e89b65706',
    'created_at': '2022-12-04T03:54:25.745409617Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': 'dcf3a2e5-4230-48aa-be2c-a9d241f170e8',
    'legs': None,
    'limit_price': None,
    'notional': None,
    'order_class': '',
    'order_type': 'market',
    'qty': '16.612273042',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'buy',
    'source': None,
    'status': 'pending_new',
    'stop_price': None,
    'submitted_at': '2022-12-04T03:54:25.744959457Z',
    'subtag': None,
    'symbol': 

### Submit Sell Order ###

In [15]:
# Create a market order to sell 1 Bitcoin
order_sell = api.submit_order('BTCUSD', qty=1, side='sell',
                             time_in_force='gtc')
value_sold = sell_qty_per_trade * btc_price
#Display Transaction details
print(f'Updated Position: {position_qty} / Value sold from position: $ {value_sold} / Account Equity is now : ${equity} Transation details')
print(f'Order Sell Transation details:')
order_sell

Updated Position: 16.570742359 / Value sold from position: $ 84791.90000000001 / Account Equity is now : $939057.4630876613 Transation details
Order Sell Transation details:


Order({   'asset_class': 'crypto',
    'asset_id': '276e2673-764b-4ab6-a611-caf665ca6340',
    'canceled_at': None,
    'client_order_id': 'df78a907-a81f-431f-97ba-fdc9cba0632a',
    'created_at': '2022-12-04T03:54:26.795005836Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': '9950debb-80bf-4372-9445-499b36816db4',
    'legs': None,
    'limit_price': None,
    'notional': None,
    'order_class': '',
    'order_type': 'market',
    'qty': '1',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'sell',
    'source': None,
    'status': 'pending_new',
    'stop_price': None,
    'submitted_at': '2022-12-04T03:54:26.793201466Z',
    'subtag': None,
    'symbol': 'BTC/USD',
    'time_in_force': 'gtc',
    'trail_percent': None,
    'trail_price': None,
    'type': 'market',
    'updated_at': '2022-12-04T03:54:26.795068226Z'})

### Check account Status ###

In [16]:
# Create an account balance variable
# get our account data from alpaca 
account = api.get_account()
beginning_deposit = float(1000000.00)
portfolio_value = float(account.portfolio_value)

    #check to see if we are restricted
if account.trading_blocked:
    print('Account is currently restricted from trading.')

    #print current market value of account
    print('Total market value is ${account.portfolio_value}.')
    
    #calculate how much profit we have.
if portfolio_value >= beginning_deposit:

    profit = portfolio_value - beginning_deposit
 
    print(f"The profit from your portfolio is: ${profit}. Your current portfolio value is ${portfolio_value}")
        
else:
    print(f"There are no profits to withdraw, your current portfolio value is $ {portfolio_value} with a net loss of -${beginning_deposit - portfolio_value} ")

There are no profits to withdraw, your current portfolio value is $ 938300.2986497408 with a net loss of -$61699.701350259245 


#pip install alpaca-py
from alpaca.data.timeframe import TimeFrame
from alpaca.data.requests import StockBarsRequest
from alpaca.data.historical import StockHistoricalDataClient

import os
import pandas as pd


client = StockHistoricalDataClient(API_KEY, API_SECRET)
request_params = StockBarsRequest(
                        symbol_or_symbols=["AAPL", "TSLA"],
                        timeframe=TimeFrame.Day,
                        start="2022-10-01 00:00:00",
                        end="2022-10-31 00:00:00"
                 )
bars = client.get_stock_bars(request_params)
bars_df = bars.df
bars_df.plot()

In [19]:
from datetime import datetime, timedelta
import math
import time

SYMBOL = 'BTCUSD'
SMA_FAST = short_window
SMA_SLOW = long_window
order_value= float(order_value)





# Description is given in the article
def get_pause():
    now = datetime.now()
    next_min = now.replace(second=0, microsecond=0) + timedelta(minutes=1)
    pause = math.ceil((next_min - now).seconds)
    print(f"Sleep for {pause}")
    return pause

# Same as the function in the random version
def get_position(symbol):
    positions = api.list_positions()
    for p in positions:
        if p.symbol == symbol:
            return float(p.qty)
    return 0

# Returns a series with the moving average
def get_sma(series, periods):
    return series.rolling(periods).mean()

# Checks whether we should buy (fast ma > slow ma)
def get_signal(fast, slow):
    print(f"Fast {fast[-1]}  /  Slow: {slow[-1]}")
    return fast[-1] > slow[-1]

# Get up-to-date 1 minute data from Alpaca and add the moving averages
def get_bars(symbol):
    bars = api.get_crypto_bars(symbol, TimeFrame.Minute).df
    bars = bars[bars.exchange == 'CBSE']
    bars[f'sma_fast'] = get_sma(bars.close, SMA_FAST)
    bars[f'sma_slow'] = get_sma(bars.close, SMA_SLOW)
    return bars

while True:
    # GET DATA
    bars = get_bars(symbol=SYMBOL)
    # CHECK POSITIONS
    position = get_position(symbol=SYMBOL)
    order_value= float(order_value)
    non_marginable_buying_power= float(non_marginable_buying_power)
    position_value = position_qty * float(btc_price)
    should_buy = btc_signals_df.loc[(btc_signals_df['Actual Returns'] >= 0), 'Signal'] = 1
    should_sell = btc_signals_df.loc[(btc_signals_df['Actual Returns'] < 0), 'Signal'] = -1
    print(f"Position: {position} / Should Buy: {should_buy}")
    if order_value >= non_marginable_buying_power and should_buy == True:
        # WE BUY ONE BITCOIN
        api.submit_order(SYMBOL, qty=QTY_PER_TRADE, side='buy',
                        time_in_force='gtc')
        print(f'Symbol: {SYMBOL} / Side: BUY / Quantity: {QTY_PER_TRADE}')
    elif order_value >= position_value and should_sell == True:
        # WE SELL ONE BITCOIN
        api.submit_order(SYMBOL, qty=QTY_PER_TRADE, side='sell',
                        time_in_force='gtc')
        print(f'Symbol: {SYMBOL} / Side: SELL / Quantity: {QTY_PER_TRADE}')
        
    else:
        print((f'Symbol: {SYMBOL} / Side: Hold'))
        
        

    time.sleep(get_pause())
    print("*"*20)

Position: 48.712227077 / Should Buy: 1
Symbol: BTCUSD / Side: Hold
Sleep for 33
********************
Position: 48.712227077 / Should Buy: 1
Symbol: BTCUSD / Side: Hold
Sleep for 58
********************
Position: 48.712227077 / Should Buy: 1
Symbol: BTCUSD / Side: Hold
Sleep for 58
********************
Position: 48.712227077 / Should Buy: 1
Symbol: BTCUSD / Side: Hold
Sleep for 58
********************
Position: 48.712227077 / Should Buy: 1
Symbol: BTCUSD / Side: Hold
Sleep for 59
********************
Position: 48.712227077 / Should Buy: 1
Symbol: BTCUSD / Side: Hold
Sleep for 58
********************
Position: 48.712227077 / Should Buy: 1
Symbol: BTCUSD / Side: Hold
Sleep for 58
********************
Position: 48.712227077 / Should Buy: 1
Symbol: BTCUSD / Side: Hold
Sleep for 59
********************
Position: 48.712227077 / Should Buy: 1
Symbol: BTCUSD / Side: Hold
Sleep for 58


KeyboardInterrupt: 

#### Liquidate position ###

In [20]:
# Get current position on Bitcoin. Yields error if we don't hold any
api.get_position('BTCUSD')

# Get all positions and find out if we hold any Bitcoin
positions = api.list_positions()
position_qty = 0
for p in positions:
    if p.symbol == 'BTCUSD':
        position_qty = float(p.qty)

# Create a market order to sell 1 Bitcoin
value_liquidated = position_qty * btc_price
liquidate_position = api.submit_order('BTCUSD', qty=position_qty, side='sell',
                             time_in_force='gtc')


#Display Transaction details
print(f'Updated Position: {position_qty} / Value Liqudated from position: $ {value_liquidated} / Account Equity is now : ${equity} Transation details')
print(f'Transation details:')
liquidate_position

Updated Position: 48.712227077 / Value Liqudated from position: $ 826080.4574180554 / Account Equity is now : $939057.4630876613 Transation details
Transation details:


Order({   'asset_class': 'crypto',
    'asset_id': '276e2673-764b-4ab6-a611-caf665ca6340',
    'canceled_at': None,
    'client_order_id': '1938efc8-cb57-462f-920e-1b5faded242b',
    'created_at': '2022-12-04T04:06:34.387530788Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': 'ce27aa22-de62-4bde-87a9-be75d7ebcf2e',
    'legs': None,
    'limit_price': None,
    'notional': None,
    'order_class': '',
    'order_type': 'market',
    'qty': '48.712227077',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'sell',
    'source': None,
    'status': 'pending_new',
    'stop_price': None,
    'submitted_at': '2022-12-04T04:06:34.384905859Z',
    'subtag': None,
    'symbol': 'BTC/USD',
    'time_in_force': 'gtc',
    'trail_percent': None,
    'trail_price': None,
    'type': 'market',
    'updated_at': '2022-12-04T04:06:34.38